In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import os
print(os.listdir("../input"))

In [ ]:
from fastai.text import *
from fastai.imports import *
from fastai.text import *
from fastai import *

In [ ]:
test = pd.read_csv('/kaggle/input/updated_test.csv')
train = pd.read_csv('/kaggle/input/updated_train.csv')

In [ ]:
train.head(5)

In [ ]:
test.head(5)

In [ ]:
train['text'][3]

In [ ]:
test.shape

# method-1 (fastai+bert)

In [ ]:
c=test['ID']
del train['ID']
del test['ID']

In [ ]:
train['target'].value_counts()

In [ ]:
train.isnull().sum(),test.isnull().sum()

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)   
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)   
    text = re.sub(r'www.[^ ]+', '', text)  
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)  
    text = re.sub(r'[^a-zA-Z]', ' ', text)   
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

train['text'] = train['text'].apply(clean_text)
test['text'] = test['text'].apply(clean_text)

In [ ]:
def random_seed(seed_value):
    import random 
    random.seed(seed_value)  
    import numpy as np
    np.random.seed(seed_value)  
    import torch
    torch.manual_seed(seed_value)  
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True   
        torch.backends.cudnn.benchmark = False

In [ ]:
path = Path('../input/')
path.ls()

In [ ]:
from sklearn.metrics import accuracy_score 
y_pred_totcb = []
c=[]
from sklearn.model_selection import KFold, RepeatedKFold
fold = KFold(n_splits=15, shuffle=True, random_state=0)
i=1

for train_index, test_index in fold.split(train):
    
    train_df = train.iloc[train_index]
    valid_df = train.iloc[test_index]

    random_seed(10)
    
    data_lm = TextLMDataBunch.from_df(Path(path), train_df, valid_df, test, text_cols=[0], bs=32)
    data_clas = TextClasDataBunch.from_df(Path(path), train_df, valid_df, test, text_cols=[0], label_cols=1, bs=32)
    
    learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.4, model_dir='/tmp/model/')
    learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))
    learn.unfreeze()
    learn.fit_one_cycle(9, 1e-3, moms=(0.8,0.7))
    learn.save_encoder('model_enc')
    
    learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.4, model_dir='/tmp/model/')
    learn.load_encoder('model_enc')
    learn.fit_one_cycle(1, 1e-2, moms=(0.8, 0.7))
    learn.freeze_to(-2)
    learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))
    learn.freeze_to(-3)
    learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))
    learn.unfreeze()
    learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))
   
    log_preds, test_labels = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
    preds = np.argmax(log_preds, 1)
    c.append(log_preds)
    y_pred_totcb.append(preds)
    print(f'fold {i} completed')
    i = i+1

In [ ]:
y_pred_totcb

In [ ]:
df = pd.DataFrame()
for i in range(15):
    col_name = 'SECTION_' + str(i)
    df[col_name] =  y_pred_totcb[i]

In [ ]:
df.shape

In [ ]:
sub = pd.DataFrame()
sub['SECTION'] = df.mode(axis=1)[0]
sub.tail()

In [ ]:
sub['SECTION'].value_counts()

In [ ]:
submission = pd.DataFrame({
    "target": sub['SECTION']
    })
submission.to_csv('./z5.csv', index=False)
print(submission)

# method-2 (tfidf)

In [ ]:

from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('english')

In [ ]:

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(train['text'], train['target'], test_size=0.30, stratify=train['target'], 
                                                random_state=1)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), max_df=1.0, 
                             min_df=3, max_features=None, binary=False, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

In [ ]:
X_train_tfidf = tfidf_vect.fit_transform(X_train)
X_cv_tfidf = tfidf_vect.transform(X_cv)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
sgd = LogisticRegression(penalty='l2', l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, verbose=0, n_jobs=None, random_state=None,  class_weight=None, warm_start=False )
ovr = OneVsRestClassifier(sgd)
ovr.fit(X_train_tfidf, y_train)
y_pred_class = ovr.predict(X_cv_tfidf)
print('f1_score       :', f1_score(y_cv, y_pred_class, average='macro'))
print('accuracy score :', accuracy_score(y_cv, y_pred_class))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), max_df=1.0, 
                             min_df=3, max_features=None, binary=False, norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)

In [ ]:

full_text = list(train['text'].values) + list(test['text'].values)
tfidf_vect.fit(full_text)

X_train_tfidf = tfidf_vect.transform(train['text'])
X_test_tfidf = tfidf_vect.transform(test['text'])

y_train = train['target']

In [ ]:

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [ ]:

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15, fit_intercept=True, max_iter=1000, tol=0.001, shuffle=True, verbose=0, epsilon=0.1, n_jobs=None, random_state=None, learning_rate='optimal', eta0=0.0, power_t=0.5, early_stopping=False, validation_fraction=0.1, n_iter_no_change=5, class_weight=None, warm_start=False, average=False)
ovr = OneVsRestClassifier(sgd)
ovr.fit(X_train_tfidf, y_train)
y_pred_class = ovr.predict_proba(X_test_tfidf)
y_pred_class

In [ ]:
d=[]
for i in y_pred_class:
    d.append(max(i))
d

In [ ]:
test['target'] = d
#test.drop(['text'], axis=1,inplace=True)
test.head()

In [ ]:

test['target'].value_counts()

In [ ]:

test.to_csv('submission6.csv', index=False)

# method-3 (Robert)

In [ ]:
!pip install simpletransformers

In [ ]:
sample_sub = pd.read_csv('/kaggle/input/updated_ss.csv')

In [ ]:
ID, target = sample_sub.columns.tolist()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import *

from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax

from simpletransformers.classification.classification_model import ClassificationModel
from sklearn.metrics import mean_squared_error as mse

In [ ]:
test.isnull().sum()

In [ ]:
def clean_text(text):
    c=['corona','coronavirus','covid','covid19','covid-19','sarscov2','19']
    for i in c:
        text=text.replace(i, '')
    return text

train['text'] = train['text'].apply(clean_text)
test['text'] = test['text'].apply(clean_text)

In [ ]:
def get_model(model_type, model_name, n_epochs = 2, train_batch_size = 112, eval_batch_size = 144, seq_len = 134, lr = 2e-5):
  model = ClassificationModel(model_type, model_name,num_labels=1, args={'train_batch_size':train_batch_size,
                                                                         "eval_batch_size": eval_batch_size,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': n_epochs,
                                                                         'max_seq_length': seq_len,
                                                                         'regression': True,
                                                                         'manual_seed': 2,
                                                                         "learning_rate":lr,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,})
  return model


In [ ]:

tmp = pd.DataFrame()
tmp['text'] = train['text']
tmp['labels'] = train['target']
tmp_test = test[['text']].rename({'text': 'text'}, axis=1)
tmp_test['labels'] = 0
tmp_trn, tmp_val = train_test_split(tmp, test_size=0.15, random_state=2)


In [ ]:

model = get_model('roberta', 'roberta-base', n_epochs=3)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, 0, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, 0, 1)
pv_1 = preds_val
pt_1 = test_preds

In [ ]:
'''
model = get_model('roberta', 'deepset/roberta-base-squad2-covid', n_epochs=3)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, 0, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, 0, 1)
pv_5 = preds_val
pt_5 = test_preds
'''

In [ ]:
'''
model = get_model('distilbert', 'distilbert-base-uncased', n_epochs=1, train_batch_size=16, eval_batch_size=16)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, 0, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, 0, 1)
pv_2 = preds_val
pt_2 = test_preds
'''

In [ ]:
model = get_model('roberta', 'roberta-large', n_epochs=1, train_batch_size=16, eval_batch_size=16)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, 0, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, 0, 1)
pv_2 = preds_val
pt_2 = test_preds

In [ ]:
model = get_model('roberta', 'roberta-large', n_epochs=2, train_batch_size=16, eval_batch_size=16, lr = 2e-5)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, 0, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, 0, 1)
pv_3 = preds_val
pt_3 = test_preds

In [ ]:
model = get_model('roberta', 'roberta-large', n_epochs=3, train_batch_size=16, eval_batch_size=16, lr = 1e-5)
model.train_model(tmp_trn)
preds_val = model.eval_model(tmp_val)[1]
preds_val = np.clip(preds_val, 0, 1)
print(f"RMSE: {mse(tmp_val['labels'], preds_val)**0.5}")
test_preds = model.eval_model(tmp_test)[1]
test_preds = np.clip(test_preds, 0, 1)
pv_4 = preds_val
pt_4 = test_preds

In [ ]:
pv = ((pv_1 * 0.3 + pv_2 * 0.7) * 0.3 + pv_3*0.7)*0.65 + pv_4*0.35
print(f"RMSE: {mse(tmp_val['labels'], pv)**0.5}")

In [ ]:
tp = ((pt_1 * 0.3 + pt_2 * 0.7) * 0.3 + pt_3*0.7)*0.65 + pt_4*0.35

In [ ]:
#tp = ((pt_2 * 0.3 + pt_3 * 0.7) * 0.3 + pt_1*0.7)*0.65 + pt_5*0.35

In [ ]:
pd.Series(tp).describe()

In [ ]:

final_test_preds = pt_1
preds_df_final = test[[ID]]
preds_df_final[target] = final_test_preds
SUB_FILE_NAME = 'roberta_ensemble15.csv'
preds_df_final.to_csv(SUB_FILE_NAME, index=False)

In [ ]:

preds_df_final.head()